In [10]:
from pyspark.sql.functions import col, explode, split, regexp_replace
from pyspark.sql.functions import col, split, explode, regexp_replace


from pyspark.sql.functions import lit, col, when
from pyspark.sql.types import StringType
from pyspark.ml.feature import FeatureHasher
import os
from pyspark.sql.functions import col, when, lit, isnan
from pyspark.sql.functions import lit, nanvl, isnan, when, avg
import category_encoders as ce
import pandas as pd
import itertools
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from category_encoders import CatBoostEncoder
import pickle
from pyspark.sql.functions import pandas_udf, PandasUDFType
from pyspark.sql.types import DoubleType, ArrayType
import numpy as np
import pandas as pd
import sklearn
sklearn.set_config(transform_output='pandas')
from pyspark.sql.functions import array
from pyspark.sql import functions as F
from pyspark.sql import SparkSession, types as T
from pyspark.sql.functions import udf
from ast import literal_eval

from pyspark.sql import DataFrame
import os
from pyspark.sql import functions as sf
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, DoubleType
import pyspark

import os


import pyspark

from pyspark.sql import SparkSession
from pyspark.sql.types import StringType, BooleanType


ram = 16

spark = SparkSession. \
    builder. \
    appName('spark_first_run') \
    .config("spark.executor.memory", f"{ram}g") \
    .config("spark.driver.maxResultSize", f"{ram}g") \
    .config("spark.driver.memory", f"{ram}g") \
    .config("spark.executor.memoryOverhead", f"{ram}g") \
    .config("spark.sql.debug.maxToStringFields", "100000") \
    .getOrCreate()


#    .config("spark.driver.memory", "10g") \

#    .config("spark.executor.memoryOverhead", "10g") \




# Register Sedona functions to Spark






from scipy.stats import t

from pyspark.sql.functions import udf, lit

import pandas as pd
from ast import literal_eval
import json
from pyspark.sql.functions import udf
import pandas as pd
import shutil
from pyspark.sql.functions import udf
from pyspark.sql.functions import udf, explode
from pyspark.sql.types import ArrayType, StructType, StructField, FloatType
from pyspark.sql.functions import desc
from time import sleep
from pyspark.sql import Row
#from pyspark.sql.functions import min, max
from pyspark.sql.functions import col
from pyspark.sql.functions import split, explode, lower, trim
from pyspark.sql.functions import when, count, col
from pyspark.sql.functions import row_number
from pyspark.sql.window import Window
from pyspark.sql.functions import monotonically_increasing_id
from pyspark.sql.functions import monotonically_increasing_id, concat, col, lit, cast
from pyspark.sql.functions import collect_list, udf
from pyspark.sql.types import DoubleType
from pyspark.sql.functions import initcap
import re
import subprocess
from pyspark.sql.functions import col, sum
from pyspark.sql.window import Window
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType
from pyspark.sql.functions import first, last

from datetime import datetime, timedelta

#### Задача 1

Выгрузить всех клиентов с суммарным весом покупок более 2 кг за последний месяц. 
Tabl A: Purchases
client_id | date (в формате yyy-mm-dd) | sku (id товара) | cnt (количество штук)
Tabl B: products
sku (id товара) | weight (вес к кг)


In [4]:
# Пример данных
data_purchases = [
    (1, "2024-01-05", 101, 2),
    (1, "2024-01-10", 102, 3),
    (2, "2024-01-15", 103, 1),
    (2, "2024-02-01", 101, 10),  
    (3, "2024-02-01", 102, 5) 
]

data_products = [
    (101, 0.5),
    (102, 0.8),
    (103, 1.2)
]

# Создаем схему для данных
schema_purchases = StructType([
    StructField("client_id", IntegerType(), True),
    StructField("date", StringType(), True),
    StructField("sku", IntegerType(), True),
    StructField("cnt", IntegerType(), True)
])

schema_products = StructType([
    StructField("sku", IntegerType(), True),
    StructField("weight", FloatType(), True)
])

# Создаем DataFrame из данных и применяем схему
df_purchases = spark.createDataFrame(data_purchases, schema=schema_purchases)
df_products = spark.createDataFrame(data_products, schema=schema_products)

# Преобразуем столбец с датой в формат DateType
df_purchases = df_purchases.withColumn("date", col("date").cast("date"))

# Определяем дату начала последнего месяца
start_date = "2024-02-01"  # начало текущего месяца
start_date_str = start_date

# Вычисляем суммарный вес покупок за последний месяц
df_total_weight = df_purchases.join(df_products, df_purchases["sku"] == df_products["sku"]) \
    .filter(col("date") >= start_date_str) \
    .groupBy("client_id") \
    .agg(sum(col("weight") * col("cnt")).alias("total_weight"))

# Фильтруем только те записи, у которых суммарный вес больше 2 кг
df_result = df_total_weight.filter(col("total_weight") > 2)

# Выводим результат
df_result.show()


+---------+------------+
|client_id|total_weight|
+---------+------------+
|        3|         4.0|
|        2|         5.0|
+---------+------------+


#### Задача 2
Дана таблица MONTHLY_REVENUE - выручка за месяц по абонентам по видам услуг:
abon_id - ID абонента
service_id - вид услуги (смс, звонки, доп сервисы)
revenue - выручка
Отфильтруйте таблицу, оставив для каждого абонента только 3 услуги с максимальной выручкой.


In [5]:
# Создаем тестовые данные
data = [
    (1, 101, 100),  # абонент 1, услуга 101, выручка 100
    (1, 102, 150),
    (1, 103, 80),
    (1, 104, 200),
    (2, 101, 120),
    (2, 102, 130),
    (2, 103, 140),
    (2, 104, 110),
    (3, 101, 90),
    (3, 102, 100),
    (3, 103, 80),
    (3, 104, 120)
]

# Создаем DataFrame
schema = ["abon_id", "service_id", "revenue"]
MONTHLY_REVENUE = spark.createDataFrame(data, schema=schema)

# Регистрируем таблицу во временном представлении
MONTHLY_REVENUE.createOrReplaceTempView("MONTHLY_REVENUE")

# SQL Select query
spark.sql('''
        SELECT abon_id, service_id, revenue
        FROM (
            SELECT abon_id, service_id, revenue,
                   ROW_NUMBER() OVER (PARTITION BY abon_id ORDER BY revenue DESC) as rn
            FROM MONTHLY_REVENUE
            ) ranked
        WHERE rn <= 3;
''') \
    .show()

+-------+----------+-------+
|abon_id|service_id|revenue|
+-------+----------+-------+
|      1|       104|    200|
|      1|       102|    150|
|      1|       101|    100|
|      2|       103|    140|
|      2|       102|    130|
|      2|       101|    120|
|      3|       104|    120|
|      3|       102|    100|
|      3|       101|     90|
+-------+----------+-------+


#### Задача 3
Найти тип первой и тип последней операции для каждого клиента и вывести их с их датами.
В данном примере это «Оплата штрафов» от 2019-12-31 и «Оплата штрафов» от 2020-03-02.
Желательно использовать один запрос без подзапросов с двумя единственными оконными функциями – по одной на максимум и минимум.


In [15]:
df = spark.read.csv("data_ex1.csv", sep=";", encoding="UTF-8", header=True)
# Показываем схему данных
df.printSchema()
df.show()

root
 |-- CUSTOMER_RK: string (nullable = true)
 |-- OPERATION_DTTM: string (nullable = true)
 |-- OPERATION_TYPE_CD: string (nullable = true)

+-----------+-------------------+--------------------+
|CUSTOMER_RK|     OPERATION_DTTM|   OPERATION_TYPE_CD|
+-----------+-------------------+--------------------+
|       2001|2019-12-31 12:12:01|      Оплата штрафов|
|       2001|2020-01-10 20:48:58|                NULL|
|       2001|2020-01-13 09:14:05|Покупка товаров и...|
|       2001|2020-01-20 17:43:51|   Оплата мобильного|
|       2001|2020-01-29 01:26:45|   Оплата мобильного|
|       2001|2020-01-31 12:02:31|Покупка товаров и...|
|       2001|2020-02-09 05:32:52|Покупка товаров и...|
|       2001|2020-02-13 08:37:38|Переводы между св...|
|       2001|2020-02-15 14:39:56|Покупка товаров и...|
|       2001|2020-02-24 12:42:16|   Оплата мобильного|
|       2001|2020-03-02 16:59:59|      Оплата штрафов|
+-----------+-------------------+--------------------+


In [18]:
# df_pandas = df.toPandas()
# df_pandas
schema = ["abon_id", "OPERATION_DTTM", "OPERATION_TYPE_CD"]

# Создаем временное представление для DataFrame
df.createOrReplaceTempView("operations")

In [21]:


# SQL Select query
spark.sql('''
    SELECT
        CUSTOMER_RK,
        FIRST_VALUE(OPERATION_TYPE_CD) OVER (PARTITION BY CUSTOMER_RK ORDER BY OPERATION_DTTM) AS FIRST_OPERATION_TYPE,
        FIRST_VALUE(OPERATION_DTTM) OVER (PARTITION BY CUSTOMER_RK ORDER BY OPERATION_DTTM) AS FIRST_OPERATION_DATE,
        LAST_VALUE(OPERATION_TYPE_CD) OVER (PARTITION BY CUSTOMER_RK ORDER BY OPERATION_DTTM ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) AS LAST_OPERATION_TYPE,
        LAST_VALUE(OPERATION_DTTM) OVER (PARTITION BY CUSTOMER_RK ORDER BY OPERATION_DTTM ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) AS LAST_OPERATION_DATE
    FROM
        operations;
''') \
    .show()


+-----------+--------------------+--------------------+-------------------+-------------------+
|CUSTOMER_RK|FIRST_OPERATION_TYPE|FIRST_OPERATION_DATE|LAST_OPERATION_TYPE|LAST_OPERATION_DATE|
+-----------+--------------------+--------------------+-------------------+-------------------+
|       2001|      Оплата штрафов| 2019-12-31 12:12:01|     Оплата штрафов|2020-03-02 16:59:59|
|       2001|      Оплата штрафов| 2019-12-31 12:12:01|     Оплата штрафов|2020-03-02 16:59:59|
|       2001|      Оплата штрафов| 2019-12-31 12:12:01|     Оплата штрафов|2020-03-02 16:59:59|
|       2001|      Оплата штрафов| 2019-12-31 12:12:01|     Оплата штрафов|2020-03-02 16:59:59|
|       2001|      Оплата штрафов| 2019-12-31 12:12:01|     Оплата штрафов|2020-03-02 16:59:59|
|       2001|      Оплата штрафов| 2019-12-31 12:12:01|     Оплата штрафов|2020-03-02 16:59:59|
|       2001|      Оплата штрафов| 2019-12-31 12:12:01|     Оплата штрафов|2020-03-02 16:59:59|
|       2001|      Оплата штрафов| 2019-

#### Задача 4
Для таблицы из п.3 найти для каждой операции количество дней с даты последней оплаты мобильного (операция "Оплата мобильного"). Если тип операции «Оплата мобильного», то кол-во дней равно 0.
Т.е. для 2020-02-15 последняя операция «Оплата мобильного» была 2020-01-29, т.е. кол-во дней составит 17 дней


In [22]:
result = spark.sql('''
    SELECT
        OPERATION_DTTM,
        OPERATION_TYPE_CD,
        CASE
            WHEN OPERATION_TYPE_CD = 'Оплата мобильного' THEN 0
            ELSE datediff(max(CASE WHEN OPERATION_TYPE_CD = 'Оплата мобильного' THEN OPERATION_DTTM END) OVER (), OPERATION_DTTM)
        END AS DAYS_SINCE_LAST_MOBILE_PAYMENT
    FROM
        operations;
''')

result.show()

+-------------------+--------------------+------------------------------+
|     OPERATION_DTTM|   OPERATION_TYPE_CD|DAYS_SINCE_LAST_MOBILE_PAYMENT|
+-------------------+--------------------+------------------------------+
|2019-12-31 12:12:01|      Оплата штрафов|                            55|
|2020-01-10 20:48:58|                NULL|                            45|
|2020-01-13 09:14:05|Покупка товаров и...|                            42|
|2020-01-20 17:43:51|   Оплата мобильного|                             0|
|2020-01-29 01:26:45|   Оплата мобильного|                             0|
|2020-01-31 12:02:31|Покупка товаров и...|                            24|
|2020-02-09 05:32:52|Покупка товаров и...|                            15|
|2020-02-13 08:37:38|Переводы между св...|                            11|
|2020-02-15 14:39:56|Покупка товаров и...|                             9|
|2020-02-24 12:42:16|   Оплата мобильного|                             0|
|2020-03-02 16:59:59|      Оплата штра